
# HW02: 版網頁爬蟲作業

本作業：從 TechNews 科技新報爬取多頁新聞標題、連結、發佈時間與文章摘要，最後存成 CSV 檔案。

使用技術：requests、BeautifulSoup、pandas、內文擷取、CSV 存檔。



## 套件安裝

請先安裝必要套件：

```bash
pip install requests beautifulsoup4 pandas
```


In [1]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [2]:

# 設定
base_url = 'https://technews.tw/page/{}'
headers = {'User-Agent': 'Mozilla/5.0'}
data = []

for page in range(1, 4):
    url = base_url.format(page)
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.find_all('article')
        
        for article in articles:
            title_tag = article.find('h1', class_='entry-title')
            time_tag = article.find('time', class_='entry-date')
            
            if title_tag and title_tag.a:
                title = title_tag.a.text.strip()
                link = title_tag.a['href']
                publish_time = time_tag.text.strip() if time_tag else None
                
                # 進入內頁擷取內文
                content_response = requests.get(link, headers=headers)
                content_soup = BeautifulSoup(content_response.text, 'html.parser')
                content_paragraphs = content_soup.select('div.entry-content p')
                content_text = ' '.join(p.text.strip() for p in content_paragraphs[:3])  # 取前三段摘要
                
                data.append({
                    '標題': title, 
                    '連結': link, 
                    '發佈時間': publish_time, 
                    '摘要': content_text
                })
                
                time.sleep(0.5)  # 每篇文章小等待，避免連續內頁請求過快
        
        print(f"第 {page} 頁擷取完成，共擷取 {len(articles)} 筆")
    else:
        print(f"第 {page} 頁擷取失敗，狀態碼:", response.status_code)
    
    time.sleep(1)


第 1 頁擷取完成，共擷取 15 筆
第 2 頁擷取完成，共擷取 15 筆
第 3 頁擷取完成，共擷取 15 筆


In [3]:

# 整理資料
df = pd.DataFrame(data)
df.head(10)


,標題,連結,發佈時間,摘要
0,日月光半導體攜高雄五所高中辦暑期營隊，厚植學生認識先進製程與產業,https://finance.technews.tw/2025/06/16/ase-sem...,None,為因應高雄產業升級與半導體 S 廊帶政策推動，高雄市政府教育局與日月光半導體攜手合作，自 1...
1,數發部預告促進資料創新利用發展條例，機關須設資料長,https://infosecu.technews.tw/2025/06/16/govern...,None,數發部今天預告「促進資料創新利用發展條例」草案，強化輔導與獎勵機制，促進資料創新利用，助攻 ...
2,美國施壓越南與中國科技脫鉤，要求出口產品降低中國成分,https://finance.technews.tw/2025/06/16/us-push...,None,三位知情人士透露，美國在與越南的關稅談判中施壓，要求越南減少在當地組裝、並出口至美國的裝置中...
3,以伊開戰反映核武抗衡潛在風險！元大航太防衛科技 ETF 交易量增,https://finance.technews.tw/2025/06/16/nuclear...,None,以色列空襲後，緊接著伊朗還擊逾百枚飛彈，衝突持續升溫，反映局勢潛在升級風險，帶動全台唯一國防...
4,西班牙葡萄牙等地抗議過度旅遊，居民高呼「你度假我倒楣」,https://technews.tw/2025/06/16/spain-portugal-...,None,南歐多座大城市 15 日有數以千計民眾上街抗議過度旅遊，示威活動主要在西班牙巴塞隆納舉行，抗...
5,證實冥王星「自我冷卻」關鍵條件，高空藍色霧霾調節著氣候,https://technews.tw/2025/06/16/webb-space-tele...,None,冥王星被籠罩在神秘藍色霧霾之下，使這顆矮行星在太陽系中顯得獨特，而最近天文學家使用韋伯太空望...
6,台灣 GDP 維持 2.8% 第三季陰雨！國泰金估央行遇經濟衰退可能降息,https://finance.technews.tw/2025/06/16/rain/,None,國泰金今日舉行第二季台灣「經濟氣候暨金融情勢」發表會，國泰金與台大產學合作計畫協同主持人徐之...
7,金融時報：中國逾三成車企流動負債超過流動資產,https://finance.technews.tw/2025/06/16/over-30...,None,英國金融時報報導，超過三成中國上市車企的流動負債超過流動資產，車企轉向供應商施壓以維持營運資...
8,挑戰台積電？ 華為新專利「四晶片封裝」曝光，昇騰 910D 恐迎突破,https://technews.tw/2025/06/16/huawei-patent-q...,None,華為近期申請「四晶片」（quad-chiplet）封裝設計專利，可能用於下代 AI 加速器昇...
9,台美關稅協商未果，商總許舒博籲勿忽略中國、歐洲市場,https://finance.technews.tw/2025/06/16/coc-cha...,None,台美關稅談判仍在進行，商總理事長許舒博今天表示，台積電與其他高科技產業已赴美投資，台灣也擬採...


In [4]:

# 儲存成 CSV 檔案
df.to_csv('technews_data.csv', index=False, encoding='utf-8-sig')
print("已儲存為 technews_data.csv")


已儲存為 technews_data.csv



## 延伸挑戰建議

- 無法成功擷取到發布時間，因此網頁發布時間分類為內文文字。
- 擷取更多頁面與資料欄位（如作者、標籤、留言數）。
- 做關鍵字分析或文字雲。
- 自動排程每日爬取新聞，做長期新聞監測。
- 嘗試繪製熱門主題變化圖表。
